In [142]:
import numpy as np

In [143]:
import torch

In [144]:
x = torch.FloatTensor([[1,2,3,4,5,6,7],[3,5,6,7,8,4,5]])
x

tensor([[1., 2., 3., 4., 5., 6., 7.],
        [3., 5., 6., 7., 8., 4., 5.]])

In [145]:
x.unfold(0,2,2)

tensor([[[1., 3.],
         [2., 5.],
         [3., 6.],
         [4., 7.],
         [5., 8.],
         [6., 4.],
         [7., 5.]]])

In [146]:
mock_filter_1 = torch.tensor([[1.0,2,1],[2,4,5],[6,7,8]])
mock_filter_2 = torch.tensor([[1.0,6,7],[1,9,5],[4,7,8]])
mock_filter_3 = torch.tensor([[1.0,3,1],[7,0,5],[9,7,0]])
mock_filter_4 = torch.tensor([[3.0,2,4],[0,4,5],[1,7,0]])

In [147]:
mock_filter_combined = torch.stack([mock_filter_1, mock_filter_2, mock_filter_3, mock_filter_4]) 

In [148]:
mock_filter_combined = torch.stack([mock_filter_combined, mock_filter_combined], dim=1)

In [149]:
mock_filter_combined.shape

torch.Size([4, 2, 3, 3])

In [10]:
print(mock_filter_combined[0])
print(mock_filter_combined[1])
print(mock_filter_combined[2])
print(mock_filter_combined[3])

tensor([[[1., 2., 1.],
         [2., 4., 5.],
         [6., 7., 8.]],

        [[1., 2., 1.],
         [2., 4., 5.],
         [6., 7., 8.]]])
tensor([[[1., 6., 7.],
         [1., 9., 5.],
         [4., 7., 8.]],

        [[1., 6., 7.],
         [1., 9., 5.],
         [4., 7., 8.]]])
tensor([[[1., 3., 1.],
         [7., 0., 5.],
         [9., 7., 0.]],

        [[1., 3., 1.],
         [7., 0., 5.],
         [9., 7., 0.]]])
tensor([[[3., 2., 4.],
         [0., 4., 5.],
         [1., 7., 0.]],

        [[3., 2., 4.],
         [0., 4., 5.],
         [1., 7., 0.]]])


In [11]:
filter_groups = mock_filter_combined.unfold(0, 2, 2)

In [12]:
filter_groups.shape

torch.Size([2, 2, 3, 3, 2])

In [13]:
filter_groups[0].permute(3,0,1,2)

tensor([[[[1., 2., 1.],
          [2., 4., 5.],
          [6., 7., 8.]],

         [[1., 2., 1.],
          [2., 4., 5.],
          [6., 7., 8.]]],


        [[[1., 6., 7.],
          [1., 9., 5.],
          [4., 7., 8.]],

         [[1., 6., 7.],
          [1., 9., 5.],
          [4., 7., 8.]]]])

In [14]:
filter_groups[1].permute(3,0,1,2)

tensor([[[[1., 3., 1.],
          [7., 0., 5.],
          [9., 7., 0.]],

         [[1., 3., 1.],
          [7., 0., 5.],
          [9., 7., 0.]]],


        [[[3., 2., 4.],
          [0., 4., 5.],
          [1., 7., 0.]],

         [[3., 2., 4.],
          [0., 4., 5.],
          [1., 7., 0.]]]])

### Testing the activations

In [15]:
import torch.nn.functional as F

In [16]:
filters = mock_filter_combined
filters.shape

torch.Size([4, 2, 3, 3])

In [17]:
inputs = torch.randn(2,2,4,4)

In [18]:
activations = F.conv2d(inputs,filters)

In [19]:
activations.shape

torch.Size([2, 4, 2, 2])

In [20]:
inputs

tensor([[[[ 0.5908, -0.6185,  0.0603, -0.3435],
          [ 1.8600,  0.2810,  0.8614, -1.3006],
          [ 0.0362, -1.7636,  1.5000,  0.4837],
          [-0.3788, -0.3929, -0.1246,  1.5903]],

         [[ 0.4894,  0.2162, -0.1395,  0.1293],
          [-2.4103,  0.1139,  0.6268, -1.5294],
          [ 1.3939, -0.5379, -0.6493, -0.7220],
          [-1.0362, -0.6277,  0.7157,  2.4491]]],


        [[[-0.5707, -0.8208, -1.6440, -1.0179],
          [ 0.0280,  0.8828, -1.1049,  0.4310],
          [-0.6384,  0.3936, -1.1833, -1.8249],
          [-0.6089,  0.1059,  0.9521,  0.9801]],

         [[ 0.3944,  0.4962,  0.5617, -0.3041],
          [-2.5077, -0.6523,  1.1314,  0.0290],
          [-0.4707,  0.0747,  1.3213,  0.3497],
          [-2.6718,  0.0443, -0.7722,  0.2070]]]])

In [21]:
activations

tensor([[[[  7.3920, -17.9428],
          [-11.2698,  28.4790]],

         [[  4.9722,  -7.8955],
          [-10.8686,  26.0483]],

         [[  0.1427, -26.9980],
          [ -3.4922, -20.3202]],

         [[ -3.5408,  -6.7657],
          [ -8.4199,  -1.8301]]],


        [[[ -8.0844,  -8.9694],
          [-18.8407,   6.5136]],

         [[-10.0255, -22.2645],
          [ -7.7474,   9.3015]],

         [[-26.1607,   4.2010],
          [-37.3097,  -0.7171]],

         [[ -2.2831,  -4.5853],
          [ -6.5378,  -2.8298]]]])

In [22]:
groups = torch.split(activations, 2, dim=1)

In [23]:
groups

(tensor([[[[  7.3920, -17.9428],
           [-11.2698,  28.4790]],
 
          [[  4.9722,  -7.8955],
           [-10.8686,  26.0483]]],
 
 
         [[[ -8.0844,  -8.9694],
           [-18.8407,   6.5136]],
 
          [[-10.0255, -22.2645],
           [ -7.7474,   9.3015]]]]), tensor([[[[  0.1427, -26.9980],
           [ -3.4922, -20.3202]],
 
          [[ -3.5408,  -6.7657],
           [ -8.4199,  -1.8301]]],
 
 
         [[[-26.1607,   4.2010],
           [-37.3097,  -0.7171]],
 
          [[ -2.2831,  -4.5853],
           [ -6.5378,  -2.8298]]]]))

In [24]:
groups[0]

tensor([[[[  7.3920, -17.9428],
          [-11.2698,  28.4790]],

         [[  4.9722,  -7.8955],
          [-10.8686,  26.0483]]],


        [[[ -8.0844,  -8.9694],
          [-18.8407,   6.5136]],

         [[-10.0255, -22.2645],
          [ -7.7474,   9.3015]]]])

### Another test case where filtere are not divisible by the number of groups

In [25]:
mock_filter_1 = torch.tensor([[1.0,2,1],[2,4,5],[6,7,8]])
mock_filter_2 = torch.tensor([[1.0,6,7],[1,9,5],[4,7,8]])
mock_filter_3 = torch.tensor([[1.0,3,1],[7,0,5],[9,7,0]])
mock_filter_4 = torch.tensor([[3.0,2,4],[0,4,5],[1,7,0]])
mock_filter_5 = torch.tensor([[3.0,2,4],[0,4,5],[1,7,0]])

In [26]:
mock_filter_combined = torch.stack([mock_filter_1, mock_filter_2, mock_filter_3, mock_filter_4,mock_filter_5]) 

In [27]:
mock_filter_combined = torch.stack([mock_filter_combined, mock_filter_combined], dim=1)
mock_filter_combined.shape

torch.Size([5, 2, 3, 3])

In [28]:
inputs = torch.randn(2,2,4,4)

In [29]:
activations = F.conv2d(inputs,mock_filter_combined)
activations.shape

torch.Size([2, 5, 2, 2])

In [30]:
inputs

tensor([[[[ 1.2040e-01,  1.2835e+00, -1.0931e+00,  1.1262e-01],
          [ 4.7745e-01, -6.2841e-01, -1.8638e-01,  6.1376e-01],
          [-1.7930e+00,  2.4869e-01,  2.4884e+00, -6.1819e-01],
          [-3.0976e+00,  1.6988e+00, -1.1210e+00, -2.5258e+00]],

         [[-1.8697e-01,  4.2179e-01,  6.0392e-01,  1.2100e+00],
          [-1.4698e+00, -4.9538e-02,  3.4695e-02, -4.1113e-01],
          [-7.7259e-02, -1.5277e+00,  1.7239e-01, -6.9664e-01],
          [ 1.1639e+00,  6.9410e-01,  4.0301e-01, -5.4661e-01]]],


        [[[ 1.1721e-01, -3.7074e-01,  9.2148e-01,  1.0557e+00],
          [ 1.9315e+00, -2.8848e-04,  6.5109e-01, -7.0157e-01],
          [ 5.1592e-01, -1.7769e+00, -8.5906e-01, -7.1418e-01],
          [-1.6302e+00,  4.7926e-01,  7.6918e-01, -1.1691e+00]],

         [[-6.7866e-01, -1.1308e+00,  2.4452e-01,  7.1639e-01],
          [-3.7630e-01,  9.4472e-02,  4.0208e-01,  8.6768e-01],
          [-9.0827e-01, -4.7403e-01, -1.7463e-01, -1.1392e+00],
          [ 1.8399e+00,  1.4815e

In [31]:
activations

tensor([[[[ -1.4883,   1.5331],
          [  1.3515, -14.5159]],

         [[  3.7410,   9.9899],
          [ -2.9278,  -4.1096]],

         [[-28.9301,   4.9425],
          [ -3.6189,   0.0523]],

         [[-13.0393,  27.1812],
          [ 18.0646,  -0.0906]],

         [[-13.0393,  27.1812],
          [ 18.0646,  -0.0906]]],


        [[[-20.0256, -27.7341],
          [  4.4577,  -2.9896]],

         [[-19.3353,  -2.7643],
          [ -0.1280,  -1.9527]],

         [[ -7.0356, -22.2357],
          [  1.2545,  -6.3595]],

         [[-10.5295,   0.4721],
          [ -0.5041,  -0.1237]],

         [[-10.5295,   0.4721],
          [ -0.5041,  -0.1237]]]])

In [32]:
groups = torch.split(activations, 2, dim=1)
list(groups)

[tensor([[[[ -1.4883,   1.5331],
           [  1.3515, -14.5159]],
 
          [[  3.7410,   9.9899],
           [ -2.9278,  -4.1096]]],
 
 
         [[[-20.0256, -27.7341],
           [  4.4577,  -2.9896]],
 
          [[-19.3353,  -2.7643],
           [ -0.1280,  -1.9527]]]]), tensor([[[[-28.9301,   4.9425],
           [ -3.6189,   0.0523]],
 
          [[-13.0393,  27.1812],
           [ 18.0646,  -0.0906]]],
 
 
         [[[ -7.0356, -22.2357],
           [  1.2545,  -6.3595]],
 
          [[-10.5295,   0.4721],
           [ -0.5041,  -0.1237]]]]), tensor([[[[-13.0393,  27.1812],
           [ 18.0646,  -0.0906]]],
 
 
         [[[-10.5295,   0.4721],
           [ -0.5041,  -0.1237]]]])]

In [33]:
groupwise_mean = [torch.mean(groups[i]) for i in range(0,2)]

In [34]:
groupwise_mean

[tensor(-4.8061), tensor(-2.5313)]

In [35]:
groupwise_std = [torch.std(groups[i]) for i in range(0,2)]
groupwise_std

[tensor(10.2343), tensor(13.4274)]

### Test channelwise receptive field calcualtions

In [36]:
def calculate_receptive_field_layer_no_batch_norm(feature_maps, linear_map_param_1, 
                                                  linear_map_param_2):
    number_of_maps = feature_maps[1]
    std_map_wise = feature_maps.transpose(0,1).contiguous().view(number_of_maps, -1).std(1)
    receptive_fields = feature_maps.transpose(1,3).div(std_map_wise).transpose(1,3)
    receptive_fields = torch.sigmoid(linear_map_param_1 * receptive_fields + linear_map_param_2)
    return receptive_fields

In [37]:
calculate_receptive_field_layer_no_batch_norm(activations)
print(activations.shape)

TypeError: calculate_receptive_field_layer_no_batch_norm() missing 2 required positional arguments: 'linear_map_param_1' and 'linear_map_param_2'

In [38]:
kk=activations.transpose(0,1)
print(kk.shape)
c =kk.contiguous().view(5, -1)

torch.Size([5, 2, 2, 2])


In [39]:
c.shape

torch.Size([5, 8])

In [40]:
B = torch.mean(c,1)
c.std(1)

tensor([11.8014,  8.3354, 11.8382, 13.5706, 13.5706])

In [41]:
print(activations)
print(activations.shape)

tensor([[[[ -1.4883,   1.5331],
          [  1.3515, -14.5159]],

         [[  3.7410,   9.9899],
          [ -2.9278,  -4.1096]],

         [[-28.9301,   4.9425],
          [ -3.6189,   0.0523]],

         [[-13.0393,  27.1812],
          [ 18.0646,  -0.0906]],

         [[-13.0393,  27.1812],
          [ 18.0646,  -0.0906]]],


        [[[-20.0256, -27.7341],
          [  4.4577,  -2.9896]],

         [[-19.3353,  -2.7643],
          [ -0.1280,  -1.9527]],

         [[ -7.0356, -22.2357],
          [  1.2545,  -6.3595]],

         [[-10.5295,   0.4721],
          [ -0.5041,  -0.1237]],

         [[-10.5295,   0.4721],
          [ -0.5041,  -0.1237]]]])
torch.Size([2, 5, 2, 2])


In [42]:
kk=activations.transpose(1,3)
print(kk)
print(kk.shape)
activations.transpose(1,3).div(torch.Tensor([0.1,1,1,1,1])).transpose(1,3)

tensor([[[[ -1.4883,   3.7410, -28.9301, -13.0393, -13.0393],
          [  1.3515,  -2.9278,  -3.6189,  18.0646,  18.0646]],

         [[  1.5331,   9.9899,   4.9425,  27.1812,  27.1812],
          [-14.5159,  -4.1096,   0.0523,  -0.0906,  -0.0906]]],


        [[[-20.0256, -19.3353,  -7.0356, -10.5295, -10.5295],
          [  4.4577,  -0.1280,   1.2545,  -0.5041,  -0.5041]],

         [[-27.7341,  -2.7643, -22.2357,   0.4721,   0.4721],
          [ -2.9896,  -1.9527,  -6.3595,  -0.1237,  -0.1237]]]])
torch.Size([2, 2, 2, 5])


tensor([[[[-1.4883e+01,  1.5331e+01],
          [ 1.3515e+01, -1.4516e+02]],

         [[ 3.7410e+00,  9.9899e+00],
          [-2.9278e+00, -4.1096e+00]],

         [[-2.8930e+01,  4.9425e+00],
          [-3.6189e+00,  5.2256e-02]],

         [[-1.3039e+01,  2.7181e+01],
          [ 1.8065e+01, -9.0637e-02]],

         [[-1.3039e+01,  2.7181e+01],
          [ 1.8065e+01, -9.0637e-02]]],


        [[[-2.0026e+02, -2.7734e+02],
          [ 4.4577e+01, -2.9896e+01]],

         [[-1.9335e+01, -2.7643e+00],
          [-1.2796e-01, -1.9527e+00]],

         [[-7.0356e+00, -2.2236e+01],
          [ 1.2545e+00, -6.3595e+00]],

         [[-1.0530e+01,  4.7210e-01],
          [-5.0414e-01, -1.2368e-01]],

         [[-1.0530e+01,  4.7210e-01],
          [-5.0414e-01, -1.2368e-01]]]])

In [43]:
receptive_fields = 2.0 * activations + 1.0
print(receptive_fields)

tensor([[[[-1.9765e+00,  4.0661e+00],
          [ 3.7031e+00, -2.8032e+01]],

         [[ 8.4820e+00,  2.0980e+01],
          [-4.8555e+00, -7.2192e+00]],

         [[-5.6860e+01,  1.0885e+01],
          [-6.2378e+00,  1.1045e+00]],

         [[-2.5079e+01,  5.5362e+01],
          [ 3.7129e+01,  8.1873e-01]],

         [[-2.5079e+01,  5.5362e+01],
          [ 3.7129e+01,  8.1873e-01]]],


        [[[-3.9051e+01, -5.4468e+01],
          [ 9.9155e+00, -4.9792e+00]],

         [[-3.7671e+01, -4.5285e+00],
          [ 7.4408e-01, -2.9053e+00]],

         [[-1.3071e+01, -4.3471e+01],
          [ 3.5089e+00, -1.1719e+01]],

         [[-2.0059e+01,  1.9442e+00],
          [-8.2822e-03,  7.5263e-01]],

         [[-2.0059e+01,  1.9442e+00],
          [-8.2822e-03,  7.5263e-01]]]])


In [44]:
receptive_fields = torch.sigmoid(receptive_fields)
print(receptive_fields)

tensor([[[[1.2169e-01, 9.8315e-01],
          [9.7595e-01, 6.6975e-13]],

         [[9.9979e-01, 1.0000e+00],
          [7.7250e-03, 7.3187e-04]],

         [[2.0228e-25, 9.9998e-01],
          [1.9503e-03, 7.5110e-01]],

         [[1.2837e-11, 1.0000e+00],
          [1.0000e+00, 6.9397e-01]],

         [[1.2837e-11, 1.0000e+00],
          [1.0000e+00, 6.9397e-01]]],


        [[[1.0971e-17, 2.2117e-24],
          [9.9995e-01, 6.8325e-03]],

         [[4.3642e-17, 1.0681e-02],
          [6.7789e-01, 5.1890e-02]],

         [[2.1050e-06, 1.3202e-19],
          [9.7094e-01, 8.1370e-06]],

         [[1.9430e-09, 8.7481e-01],
          [4.9793e-01, 6.7975e-01]],

         [[1.9430e-09, 8.7481e-01],
          [4.9793e-01, 6.7975e-01]]]])


In [45]:
receptive_fields.shape
receptive_fields= receptive_fields.transpose(1, 3).add(torch.Tensor([0.1,1,1,1,2])).transpose(1,3)

In [46]:
print(receptive_fields)

tensor([[[[0.2217, 1.0831],
          [1.0759, 0.1000]],

         [[1.9998, 2.0000],
          [1.0077, 1.0007]],

         [[1.0000, 2.0000],
          [1.0020, 1.7511]],

         [[1.0000, 2.0000],
          [2.0000, 1.6940]],

         [[2.0000, 3.0000],
          [3.0000, 2.6940]]],


        [[[0.1000, 0.1000],
          [1.1000, 0.1068]],

         [[1.0000, 1.0107],
          [1.6779, 1.0519]],

         [[1.0000, 1.0000],
          [1.9709, 1.0000]],

         [[1.0000, 1.8748],
          [1.4979, 1.6798]],

         [[2.0000, 2.8748],
          [2.4979, 2.6798]]]])


In [47]:
print(receptive_fields.shape)

torch.Size([2, 5, 2, 2])


In [48]:
groups = torch.split(activations, 2, dim=1)
groups = list(groups)
print(groups)

[tensor([[[[ -1.4883,   1.5331],
          [  1.3515, -14.5159]],

         [[  3.7410,   9.9899],
          [ -2.9278,  -4.1096]]],


        [[[-20.0256, -27.7341],
          [  4.4577,  -2.9896]],

         [[-19.3353,  -2.7643],
          [ -0.1280,  -1.9527]]]]), tensor([[[[-28.9301,   4.9425],
          [ -3.6189,   0.0523]],

         [[-13.0393,  27.1812],
          [ 18.0646,  -0.0906]]],


        [[[ -7.0356, -22.2357],
          [  1.2545,  -6.3595]],

         [[-10.5295,   0.4721],
          [ -0.5041,  -0.1237]]]]), tensor([[[[-13.0393,  27.1812],
          [ 18.0646,  -0.0906]]],


        [[[-10.5295,   0.4721],
          [ -0.5041,  -0.1237]]]])]


### Test regularizer on activations in layer

#### calculate the numerator

In [49]:
print(groups[0].shape)
print(groups[0])

torch.Size([2, 2, 2, 2])
tensor([[[[ -1.4883,   1.5331],
          [  1.3515, -14.5159]],

         [[  3.7410,   9.9899],
          [ -2.9278,  -4.1096]]],


        [[[-20.0256, -27.7341],
          [  4.4577,  -2.9896]],

         [[-19.3353,  -2.7643],
          [ -0.1280,  -1.9527]]]])


In [50]:
difference_tensor = torch.sub(groups[0], groups[0][0][0])
difference_tensor_2 = torch.sub(groups[1], groups[1][0][0])
print(difference_tensor.shape)

torch.Size([2, 2, 2, 2])


In [51]:
print(difference_tensor.norm(1))
print(difference_tensor_2.norm(1))


tensor(126.9959)
tensor(146.4753)


In [52]:
difference_tensor

tensor([[[[  0.0000,   0.0000],
          [  0.0000,   0.0000]],

         [[  5.2293,   8.4568],
          [ -4.2793,  10.4064]]],


        [[[-18.5374, -29.2672],
          [  3.1062,  11.5263]],

         [[-17.8470,  -4.2973],
          [ -1.4795,  12.5633]]]])

In [53]:
input = torch.empty(2)
l1_norms = torch.zeros_like(input)
batch_size = 2
for i in range(0, 2):
    current_group = groups[i]
    norm = torch.zeros(1)
    
    for j in range(0, 2):
        difference_tensor = torch.sub(current_group[j], current_group[j][0])
        print(difference_tensor)
        norm = norm  + difference_tensor.norm(1)
    
    #difference_tensor = torch.sub(current_group, current_group[0][0])
    l1_norms[i] = norm

tensor([[[ 0.0000,  0.0000],
         [ 0.0000,  0.0000]],

        [[ 5.2293,  8.4568],
         [-4.2793, 10.4064]]])
tensor([[[ 0.0000,  0.0000],
         [ 0.0000,  0.0000]],

        [[ 0.6904, 24.9699],
         [-4.5857,  1.0369]]])
tensor([[[ 0.0000,  0.0000],
         [ 0.0000,  0.0000]],

        [[15.8907, 22.2387],
         [21.6835, -0.1429]]])
tensor([[[ 0.0000,  0.0000],
         [ 0.0000,  0.0000]],

        [[-3.4939, 22.7078],
         [-1.7586,  6.2359]]])


In [54]:
l1_norms

tensor([59.6546, 94.1519])

In [55]:
current_group

tensor([[[[-28.9301,   4.9425],
          [ -3.6189,   0.0523]],

         [[-13.0393,  27.1812],
          [ 18.0646,  -0.0906]]],


        [[[ -7.0356, -22.2357],
          [  1.2545,  -6.3595]],

         [[-10.5295,   0.4721],
          [ -0.5041,  -0.1237]]]])

In [56]:
current_group[0][0].shape

torch.Size([2, 2])

In [57]:
difference_tensor

tensor([[[ 0.0000,  0.0000],
         [ 0.0000,  0.0000]],

        [[-3.4939, 22.7078],
         [-1.7586,  6.2359]]])

### calculate with denominator

In [58]:
print(groups[0].shape)
print(groups)

torch.Size([2, 2, 2, 2])
[tensor([[[[ -1.4883,   1.5331],
          [  1.3515, -14.5159]],

         [[  3.7410,   9.9899],
          [ -2.9278,  -4.1096]]],


        [[[-20.0256, -27.7341],
          [  4.4577,  -2.9896]],

         [[-19.3353,  -2.7643],
          [ -0.1280,  -1.9527]]]]), tensor([[[[-28.9301,   4.9425],
          [ -3.6189,   0.0523]],

         [[-13.0393,  27.1812],
          [ 18.0646,  -0.0906]]],


        [[[ -7.0356, -22.2357],
          [  1.2545,  -6.3595]],

         [[-10.5295,   0.4721],
          [ -0.5041,  -0.1237]]]]), tensor([[[[-13.0393,  27.1812],
          [ 18.0646,  -0.0906]]],


        [[[-10.5295,   0.4721],
          [ -0.5041,  -0.1237]]]])]


In [59]:
input = torch.empty(2)
l1_norms = torch.zeros_like(input)
for i in range(0, 2):
    current_group = groups[i]
    # print(current_group.size())
    current_norm = torch.zeros(1)
    for j in range(0, 2):
        current_map_set = current_group[j]  
        print(current_map_set.shape)
        num = current_group.size(0)
        # print(num)
        sub_tensor = current_map_set.contiguous().view(num, -1)
        # print(sub_tensor.shape)
        sub_tensor_2 = current_map_set[0].view(-1,4)
        # print(sub_tensor_2.shape)
        difference_tensor = torch.sub(sub_tensor, sub_tensor_2)
        print(difference_tensor)
        num_tensor = difference_tensor.norm(1, dim=1)
        denom_tensor = torch.add(sub_tensor.norm(1, dim=1), sub_tensor_2.norm(1))
        value_tensor = torch.div(num_tensor, denom_tensor).norm(1)
        current_norm = current_norm + value_tensor
    l1_norms[i] = current_norm

torch.Size([2, 2, 2])
tensor([[ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 5.2293,  8.4568, -4.2793, 10.4064]])
torch.Size([2, 2, 2])
tensor([[ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.6904, 24.9699, -4.5857,  1.0369]])
torch.Size([2, 2, 2])
tensor([[ 0.0000,  0.0000,  0.0000,  0.0000],
        [15.8907, 22.2387, 21.6835, -0.1429]])
torch.Size([2, 2, 2])
tensor([[ 0.0000,  0.0000,  0.0000,  0.0000],
        [-3.4939, 22.7078, -1.7586,  6.2359]])


In [60]:
print(sub_tensor)

tensor([[ -7.0356, -22.2357,   1.2545,  -6.3595],
        [-10.5295,   0.4721,  -0.5041,  -0.1237]])


In [61]:
print(denom_tensor)

tensor([73.7705, 48.5147])


In [62]:
print(num_tensor)

tensor([ 0.0000, 34.1961])


In [63]:
l1_norms

tensor([1.1095, 1.3299])

In [64]:
l1_norms

tensor([1.1095, 1.3299])

In [65]:
difference_tensor

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000],
        [-3.4939, 22.7078, -1.7586,  6.2359]])

In [66]:
difference_tensor.norm(1, dim=1)

tensor([ 0.0000, 34.1961])

### Activation regularizer across a pair of layer

In [67]:
# our first set of activations are : <activations>
#create second set of activations
inputs = torch.randn(2,2,5,5)
activations_set2 = F.conv2d(inputs,mock_filter_combined)
activations_set2.shape

torch.Size([2, 5, 3, 3])

In [68]:
print(activations.shape)

torch.Size([2, 5, 2, 2])


In [69]:
maps_per_group = int(len(activations_set2[1]) / 2)
#print(maps_per_group)
activation_groups = torch.split(activations_set2, 2, dim=1)
activation_groups = list(activation_groups)

In [70]:
activation_groups

[tensor([[[[ -4.5527, -18.1321,  16.7374],
           [-28.2286, -23.0370,  -6.3070],
           [-24.6740, -32.5130, -16.3835]],
 
          [[-19.3668, -18.4149,  25.9876],
           [-19.5884, -31.2367,  -0.7901],
           [-19.3203, -58.2871,   0.1974]]],
 
 
         [[[ -0.3672, -20.0485, -32.0842],
           [ 19.9001,  -1.1771, -26.2464],
           [ 15.1241,   2.3370,  -9.4676]],
 
          [[ 11.1805,   6.6425, -32.4661],
           [ 32.9930,  -5.7652, -47.9725],
           [ 22.8655, -16.8197, -35.0720]]]]),
 tensor([[[[-12.7268,  -1.2030,   2.4297],
           [ -5.9311, -26.6071, -16.3488],
           [-33.3784,   3.0060, -41.1298]],
 
          [[-14.2059,  -4.0345,  -8.9748],
           [  6.9010, -30.6611,  11.7742],
           [-20.9535, -16.3840,  -4.6613]]],
 
 
         [[[ -4.1640,   0.6039,  -7.7638],
           [ 12.7781,  13.2537, -11.0101],
           [ -0.2555,  11.8302,  -3.2664]],
 
          [[ 14.4106,   8.2022, -20.8819],
           [ 21.5114,  -9.

In [71]:
maps_per_group = int(len(activations[1]) / 2)
#print(maps_per_group)
activation_groups_layer_1 = torch.split(activations, 2, dim=1)
activation_groups_layer_1 = list(activation_groups_layer_1)

In [72]:
activation_groups_layer_1

[tensor([[[[ -1.4883,   1.5331],
           [  1.3515, -14.5159]],
 
          [[  3.7410,   9.9899],
           [ -2.9278,  -4.1096]]],
 
 
         [[[-20.0256, -27.7341],
           [  4.4577,  -2.9896]],
 
          [[-19.3353,  -2.7643],
           [ -0.1280,  -1.9527]]]]), tensor([[[[-28.9301,   4.9425],
           [ -3.6189,   0.0523]],
 
          [[-13.0393,  27.1812],
           [ 18.0646,  -0.0906]]],
 
 
         [[[ -7.0356, -22.2357],
           [  1.2545,  -6.3595]],
 
          [[-10.5295,   0.4721],
           [ -0.5041,  -0.1237]]]]), tensor([[[[-13.0393,  27.1812],
           [ 18.0646,  -0.0906]]],
 
 
         [[[-10.5295,   0.4721],
           [ -0.5041,  -0.1237]]]])]

In [73]:
input = torch.empty(2)
groupwise_activation_norms = torch.zeros_like(input)

In [74]:
for i in range(0, 2):
    current_group = activation_groups[i]
    current_group_layer_1 = activation_groups_layer_1[i]
    
    num = current_group.size(0) * current_group.size(1)
    size_of_map = current_group.size(2) * current_group.size(3)
    size_of_map_layer_1 = current_group_layer_1.size(2) * current_group_layer_1.size(3)
    
    map_per_row_view = current_group.contiguous().view(num, -1)       # shape: (Batches*Channels) X (Height * Width)
    first_map_row_view =  current_group_layer_1[0][0].view(-1, size_of_map_layer_1)  # shape: 1 X (Height * Width)
    
    difference_tensor = torch.sub(map_per_row_view, first_map_row_view)
    numerator_tensor = difference_tensor.norm(1, dim=1)
    denominator_tensor = torch.add(map_per_row_view.norm(1, dim=1), first_map_row_view.norm(1))
    groupwise_activation_norms[i] = torch.div(numerator_tensor, denominator_tensor).sum()

RuntimeError: The size of tensor a (9) must match the size of tensor b (4) at non-singleton dimension 1

In [75]:
activation_groups[i].shape

torch.Size([2, 2, 3, 3])

In [76]:
activation_groups_layer_1[i].shape

torch.Size([2, 2, 2, 2])

In [77]:
activation_groups_layer_1[i][0][0].unsqueeze(0)

tensor([[[ -1.4883,   1.5331],
         [  1.3515, -14.5159]]])

### Comparing two neurons from different group and same groups:

In [79]:
def purely_diff_based_measure(unit_x, unit_y, feature_maps):
    
    
    

NameError: name 'purely_diff_based_measure' is not defined

In [80]:
mock_filter_combined.shape

torch.Size([5, 2, 3, 3])

In [84]:
inputs = torch.randn(2,2,4,4)

In [85]:
inputs

tensor([[[[ 0.7321, -0.2079,  1.5025, -1.2202],
          [-1.6360,  0.2292, -0.3381, -0.0331],
          [-0.1250,  1.0014, -0.8525, -0.9578],
          [ 1.3452,  1.5831,  0.1439,  0.5800]],

         [[ 0.3350,  1.7472,  0.0860, -1.7742],
          [-0.9988,  0.2592,  1.5437, -1.2707],
          [ 0.8323,  0.9461,  0.7537, -0.0528],
          [ 0.5220, -0.6177,  0.5546,  0.3272]]],


        [[[-1.5640, -0.6262,  0.4593,  0.3129],
          [-0.0613,  0.5867,  1.3038,  0.4860],
          [-0.6078,  0.0202, -0.0098,  0.4428],
          [-1.0598,  0.9279,  1.4967,  0.0815]],

         [[-0.8611,  0.4072, -0.1977,  0.1255],
          [ 0.2636, -1.1961,  2.0308, -0.4275],
          [-1.0225,  1.0798, -1.1144, -1.6496],
          [-0.4293,  2.8448, -0.2664,  0.3503]]]])

In [86]:
activations = F.conv2d(inputs,mock_filter_combined)
activations.shape

torch.Size([2, 5, 2, 2])

In [87]:
activations

tensor([[[[25.5321,  3.9110],
          [31.8075, 17.9826]],

         [[44.8827, -6.0560],
          [46.2906, 10.6101]],

         [[14.8567, 17.0462],
          [28.0564, 24.9592]],

         [[34.9558, -4.6231],
          [13.8158, -0.9317]],

         [[34.9558, -4.6231],
          [13.8158, -0.9317]]],


        [[[ 0.9630,  2.2310],
          [25.1521, 32.4894]],

         [[ 1.6677, 20.9894],
          [52.8303, 31.9127]],

         [[ 8.2962, -0.9387],
          [-2.3180, 53.6844]],

         [[13.6379,  8.4810],
          [36.4244,  6.9291]],

         [[13.6379,  8.4810],
          [36.4244,  6.9291]]]])

In [109]:
std_map_wise

tensor([[[[25.5321,  3.9110],
          [31.8075, 17.9826]],

         [[ 0.9630,  2.2310],
          [25.1521, 32.4894]]],


        [[[44.8827, -6.0560],
          [46.2906, 10.6101]],

         [[ 1.6677, 20.9894],
          [52.8303, 31.9127]]],


        [[[14.8567, 17.0462],
          [28.0564, 24.9592]],

         [[ 8.2962, -0.9387],
          [-2.3180, 53.6844]]],


        [[[34.9558, -4.6231],
          [13.8158, -0.9317]],

         [[13.6379,  8.4810],
          [36.4244,  6.9291]]],


        [[[34.9558, -4.6231],
          [13.8158, -0.9317]],

         [[13.6379,  8.4810],
          [36.4244,  6.9291]]]])

In [93]:
activations[:,4]

tensor([[[34.9558, -4.6231],
         [13.8158, -0.9317]],

        [[13.6379,  8.4810],
         [36.4244,  6.9291]]])

In [95]:
unit_x = 0
unit_y = 1
act_unit_x = activations[:, unit_x]
act_unit_y = activations[:, unit_y]

In [96]:
act_unit_x

tensor([[[25.5321,  3.9110],
         [31.8075, 17.9826]],

        [[ 0.9630,  2.2310],
         [25.1521, 32.4894]]])

In [97]:
act_unit_y

tensor([[[44.8827, -6.0560],
         [46.2906, 10.6101]],

        [[ 1.6677, 20.9894],
         [52.8303, 31.9127]]])

In [98]:
act_unit_y.shape

torch.Size([2, 2, 2])

In [100]:
number_of_maps = activations.size(1)

In [102]:
number_of_maps

5

In [110]:
std_map_wise = activations.transpose(0, 1).contiguous().view(number_of_maps, -1).std(1)

In [111]:
std_map_wise.shape

torch.Size([5])

In [114]:
activations

tensor([[[[25.5321,  3.9110],
          [31.8075, 17.9826]],

         [[44.8827, -6.0560],
          [46.2906, 10.6101]],

         [[14.8567, 17.0462],
          [28.0564, 24.9592]],

         [[34.9558, -4.6231],
          [13.8158, -0.9317]],

         [[34.9558, -4.6231],
          [13.8158, -0.9317]]],


        [[[ 0.9630,  2.2310],
          [25.1521, 32.4894]],

         [[ 1.6677, 20.9894],
          [52.8303, 31.9127]],

         [[ 8.2962, -0.9387],
          [-2.3180, 53.6844]],

         [[13.6379,  8.4810],
          [36.4244,  6.9291]],

         [[13.6379,  8.4810],
          [36.4244,  6.9291]]]])

In [120]:
receptive_fields = activations.transpose(1, 3).div(std_map_wise).transpose(1, 3)

In [122]:
receptive_fields.shape

torch.Size([2, 5, 2, 2])

In [123]:
receptive_fields

tensor([[[[ 1.9153,  0.2934],
          [ 2.3860,  1.3490]],

         [[ 2.0360, -0.2747],
          [ 2.0999,  0.4813]],

         [[ 0.8197,  0.9405],
          [ 1.5480,  1.3771]],

         [[ 2.3186, -0.3067],
          [ 0.9164, -0.0618]],

         [[ 2.3186, -0.3067],
          [ 0.9164, -0.0618]]],


        [[[ 0.0722,  0.1674],
          [ 1.8868,  2.4372]],

         [[ 0.0757,  0.9521],
          [ 2.3965,  1.4476]],

         [[ 0.4577, -0.0518],
          [-0.1279,  2.9621]],

         [[ 0.9046,  0.5625],
          [ 2.4160,  0.4596]],

         [[ 0.9046,  0.5625],
          [ 2.4160,  0.4596]]]])

In [124]:
receptive_fields = torch.sigmoid(receptive_fields)

In [284]:
receptive_fields

tensor([[[[0.8716, 0.5728],
          [0.9158, 0.7940]],

         [[0.8845, 0.4317],
          [0.8909, 0.6181]],

         [[0.6942, 0.7192],
          [0.8246, 0.7985]],

         [[0.9104, 0.4239],
          [0.7143, 0.4846]],

         [[0.9104, 0.4239],
          [0.7143, 0.4846]]],


        [[[0.5181, 0.5417],
          [0.8684, 0.9196]],

         [[0.5189, 0.7215],
          [0.9166, 0.8096]],

         [[0.6125, 0.4871],
          [0.4681, 0.9508]],

         [[0.7119, 0.6370],
          [0.9180, 0.6129]],

         [[0.7119, 0.6370],
          [0.9180, 0.6129]]]])

In [285]:
unit_x = 0
unit_y = 1
act_unit_x = receptive_fields[:, unit_x]
act_unit_y = receptive_fields[:, unit_y]

In [286]:
act_unit_y.shape

torch.Size([2, 2, 2])

In [287]:
act_unit_x

tensor([[[0.8716, 0.5728],
         [0.9158, 0.7940]],

        [[0.5181, 0.5417],
         [0.8684, 0.9196]]])

In [288]:
act_unit_y

tensor([[[0.8845, 0.4317],
         [0.8909, 0.6181]],

        [[0.5189, 0.7215],
         [0.9166, 0.8096]]])

In [213]:
difference_tensor = act_unit_x - act_unit_y

In [289]:
hadamard_prod = torch.mul(act_unit_x, act_unit_y)

In [290]:
hadamard_prod

tensor([[[0.7710, 0.2473],
         [0.8158, 0.4907]],

        [[0.2688, 0.3909],
         [0.7959, 0.7446]]])

In [214]:
difference_tensor

tensor([[[-0.0129,  0.1411],
         [ 0.0249,  0.1759]],

        [[-0.0009, -0.1798],
         [-0.0482,  0.1100]]])

In [215]:
difference_tensor.norm(1)

tensor(0.6936)

In [291]:
hadamard_prod.norm(1)

tensor(4.5250)

In [216]:
score = (2 * difference_tensor.norm(1)) / (act_unit_x.norm(1) + act_unit_y.norm(1) + difference_tensor.norm(1))

In [217]:
score

tensor(0.1111)

In [296]:
unit_x = 2
unit_y = 3
act_unit_x = receptive_fields[:, unit_x]
act_unit_y = receptive_fields[:, unit_y]
difference_tensor = act_unit_x - act_unit_y
hadamard_prod = torch.mul(act_unit_x, act_unit_y)
print(difference_tensor)

tensor([[[-0.2162,  0.2953],
         [ 0.1103,  0.3140]],

        [[-0.0994, -0.1500],
         [-0.4500,  0.3379]]])


In [297]:
difference_tensor.norm(1)

tensor(1.9731)

In [305]:
hadamard_prod

tensor([[[0.6320, 0.3049],
         [0.5890, 0.3869]],

        [[0.4360, 0.3103],
         [0.4297, 0.5828]]])

In [220]:
score = (2 * difference_tensor.norm(1)) / (act_unit_x.norm(1) + act_unit_y.norm(1) + difference_tensor.norm(1))

In [221]:
score

tensor(0.3049)

In [197]:
receptive_fields.shape

torch.Size([2, 5, 2, 2])

In [306]:
unit_x = np.array([0 ,2])
unit_y = np.array([1 ,3])

act_unit_x = receptive_fields[:, unit_x]
act_unit_y = receptive_fields[:, unit_y]

In [307]:
act_unit_x[:,0]

tensor([[[0.8716, 0.5728],
         [0.9158, 0.7940]],

        [[0.5181, 0.5417],
         [0.8684, 0.9196]]])

In [308]:
act_unit_y[:,0]

tensor([[[0.8845, 0.4317],
         [0.8909, 0.6181]],

        [[0.5189, 0.7215],
         [0.9166, 0.8096]]])

In [309]:
difference_tensor = act_unit_x - act_unit_y

In [310]:
difference_tensor[:, 0]

tensor([[[-0.0129,  0.1411],
         [ 0.0249,  0.1759]],

        [[-0.0009, -0.1798],
         [-0.0482,  0.1100]]])

In [311]:
difference_tensor[:, 1]

tensor([[[-0.2162,  0.2953],
         [ 0.1103,  0.3140]],

        [[-0.0994, -0.1500],
         [-0.4500,  0.3379]]])

In [312]:
hadamard_tensor = torch.mul(act_unit_x, act_unit_y)

In [314]:
hadamard_tensor[:,0]

tensor([[[0.7710, 0.2473],
         [0.8158, 0.4907]],

        [[0.2688, 0.3909],
         [0.7959, 0.7446]]])

In [280]:
score = []
for i in range(len(unit_x)):
    score.append((2 * difference_tensor[:,i].norm(1)) 
                 / (act_unit_x[:, i].norm(1) + act_unit_y[:, i].norm(1) + difference_tensor[:,i].norm(1)))

In [281]:
difference_tensor.shape

torch.Size([2, 2, 2, 2])

In [282]:
torch.stack(score)

tensor([0.1111, 0.3049])

In [283]:
score

[tensor(0.1111), tensor(0.3049)]